<a href="https://colab.research.google.com/github/Gallicchio-Lab/sbdd-on-a-laptop/blob/main/notebooks/4_Structure_Refinement_and_Binding_Energy_Scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop on Structure-Based Drug Design on a Laptop Spring 2026

# 4.1 Structure Refinement and Binding Energy Scoring

Based on  https://github.com/pablo-arantes/making-it-rain
https://prolif.readthedocs.io/en/stable/notebooks/md-ligand-protein.html

In this session, we will refine the structures of the protein-ligand complexes and calculate their binding energies using the [OpenMM](https://openmm.org/) molecular mechanics engine with an implicit solvent model.


To run the notebook:

1. Connect to a Google Colab Runtime by clicking on the down-arrow next to the `Connect` button on the upper left, select `Change runtime type`, and selecting a `T4` GPU. Then click `Connect`.
2. Execute the first cell to install `condacolab`. Google Colab will restart. When done, move to the following cells.
3. Execute the second cell to install the required software components then execute the third to test that the software is working.
4. At this point, you are at the `Create Working Directory and Upload Files` cell. Here enter the name of directory where the calculation will be performed. Any name that makes sense to you is fine. Avoid spaces.
    *  The cell will create the folder
    *  Using the file browser, navigate to the folder and upload the structure input files into it
    *  The workflow needs the pdb file of the protein-ligand complex we prepared in Maestro in Session 3.2
    *  It also needs the `.sdf` file of the ligand to assign force field parameters and the name of ligand residue (`LIG`, for example) assigned in Maestro.
5. Enter the names of the input files then execute the cell `Energy Refinement and Binding Energy of a Protein-Ligand Complex` to energy-minimize the complex and calculate the binding energy. Write down the value of the binding energy.
6. The following cells allows you to view the structure of the complex and the before and after the energy minimization and the protein-ligand interaction diagram
    *  Manipulate the structure to inspect the protein-ligand interactions
    *  Use the protein-ligand 2D interaction diagram as a guide and identify the corresponding interactions in the 3D cartoon
7. Repeat the above with each assigned protein-ligand complex. You don't need to create the working directory each time. Just upload the relevant files into the folder.
8. When done, use the last cell to download the results as a `.zip` file.


In [ ]:
#@title **Install Conda Colab**
#@markdown It will restart the kernel (session), don't worry.
# !pip install -q condacolab
# import condacolab
# condacolab.install()
!pip install -q condacolab
import condacolab
condacolab.install_from_url("https://github.com/conda-forge/miniforge/releases/download/25.3.1-0/Miniforge3-Linux-x86_64.sh")

In [ ]:
##@title **Install dependencies**
##@markdown It will take a few minutes, please, drink a coffee and wait. ;-)
## install dependencies
import subprocess
import sys

#fixes sys.path that gives torchvision import error
original_syspath = sys.path.copy()
print(original_syspath)
new_syspath = ['/content', '/env/python', '/usr/local/lib/python312.zip', '/usr/local/lib/python3.12', '/usr/local/lib/python3.12/lib-dynload', '/usr/local/lib/python3.12/site-packages' ]
sys.path = new_syspath

subprocess.run("mamba install openmm openmmforcefields nglview mdanalysis -y", shell=True)
subprocess.run("pip install pandamap", shell=True)

In [ ]:
#@title ### **Test the OpenMM MD Engine Installation**
import openmm.testInstallation
openmm.testInstallation.main()

## Working Directory and Input Files

1. Set the name of the working directory (`workDir`) below then upload the structure files into it
2. The workflow needs a pdb file with the protein-ligand complex and a sdf file of the ligand

In [ ]:
#@title ### **Create Working Directory and Upload Files**
import os
workDir = '/content/hpk1-score' #@param {type:"string"}
os.makedirs(workDir, exist_ok=True)

* Using the file browser on the left, upload the files into the working directory. Enter their names below.

In [ ]:
#@title ### **Energy Refinement and Binding Energy of a Protein-Ligand Complex**
from openmm import XmlSerializer, Vec3, LangevinMiddleIntegrator
from openmm.app import PDBFile, Simulation
from openmm.unit import nanometer, amu, angstrom, kelvin, picosecond, kilocalorie_per_mole
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator
from pathlib import Path

ProteinComplex_PDB_file_name = '7l25-lig2.pdb' #@param {type:"string"}
Ligand_SDF_file_name = "lig2.sdf" #@param {type:"string"}
Ligand_resname = 'LIG1' #@param {type:"string"}

cmplx_pdb = workDir + '/' + ProteinComplex_PDB_file_name
ligand_sdf = workDir + '/' + Ligand_SDF_file_name
ff_cache_file = workDir + '/ff.json'

displacement = Vec3(200.0,0,0) * angstrom

force_fields = ['amber14-all.xml','amber14/tip3p.xml','implicit/obc2.xml']

cmplx = PDBFile(cmplx_pdb)
ligmolecules = Molecule.from_file(ligand_sdf, file_format='SDF',
                                    allow_undefined_stereo=True)

system_generator = SystemGenerator(forcefields=force_fields,
                                    small_molecule_forcefield="openff-2.0.0",
                                    cache=ff_cache_file)

system=system_generator.create_system(cmplx.topology, molecules=ligmolecules)

basename = Path(ProteinComplex_PDB_file_name).stem
sysxml = workDir + '/' + basename + '_sys.xml'
syspdb = workDir + '/' + basename + '_sys.pdb'

with open(sysxml, 'w') as output:
    output.write(XmlSerializer.serialize(system))

#pdb before energy minimization
PDBFile.writeFile(cmplx.topology, cmplx.positions, open(syspdb,'w'), keepIds=True)

#energy minimization
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picosecond)
simulation = Simulation(cmplx.topology, system, integrator)
simulation.context.setPositions(cmplx.positions)
e_first = simulation.context.getState(getEnergy = True).getPotentialEnergy()
print("Potential energy before energy minimization =", e_first/kilocalorie_per_mole, " kcal/mol")
simulation.minimizeEnergy()
e_bound = simulation.context.getState(getEnergy = True).getPotentialEnergy()
print("Potential energy after energy minimization =", e_bound/kilocalorie_per_mole, " kcal/mol")

#pdb after minimization
minpos = simulation.context.getState(getPositions=True).getPositions()
syspdbmin = workDir + '/' + basename + '_min.pdb'
PDBFile.writeFile(cmplx.topology, minpos, open(syspdbmin,'w'), keepIds=True)

#find the ligand atoms
lig_atoms = []
for atom in cmplx.topology.atoms():
    if atom.residue.name == Ligand_resname:
        lig_atoms.append(atom.index)

#translate the ligand atoms far away
for i in lig_atoms:
    minpos[i] += displacement
simulation.context.setPositions(minpos)
e_unbound = simulation.context.getState(getEnergy = True).getPotentialEnergy()
print("Potential energy after displacing the ligand =", e_unbound/kilocalorie_per_mole, " kcal/mol")

print("Binding energy = ", (e_bound - e_unbound)/kilocalorie_per_mole, " kcal/mol")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

* Write down the calculated binding energy of this complex.
Binding energy =  -13.75004894190947  kcal/mol

In [ ]:
#@title ### **View the structure of the complex before and after energy minimization**
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:

import nglview as nv
# syspdb is the structure before minimization, syspdbmin is the structure after energy minimization
# nv.show_structure_file(syspdb, height='600px', default_representation=False)
view = nv.show_structure_file(syspdbmin, height='600px', default_representation=False)
view.add_cartoon("protein")
view.add_line("protein")
view.add_ball_and_stick("LIG")
view

In [ ]:
#@title ### **View the protein-ligand interaction diagram**

from pandamap import HybridProtLigMapper
from IPython.display import Image, display, HTML

# Basic initialization
mapper = HybridProtLigMapper(syspdbmin, ligand_resname="LIG")

# Run full workflow and generate 2D image
mapper.run_analysis()
interaction_png_file = workDir + '/' + basename + '_interactions.png'
mapper.visualize(output_file=interaction_png_file)


display(Image(interaction_png_file, width=800))


In [ ]:
#@title ### **Download the results as a zip file**

import zipfile
from google.colab import files

print("Downloading result files.")
filename = f'{workDir}.zip'

with zipfile.ZipFile(filename, 'w') as zip_file:
    dir_path = f'{workDir}'
    for root, directory, items in os.walk(dir_path):
        for item in items:
            path = os.path.join(root, item)
            zip_file.write(path, arcname=os.path.relpath(os.path.join(root, item), dir_path), compress_type=zipfile.ZIP_DEFLATED)

files.download(filename)